### Importing Packages

In [1]:
from pyimagesearch.io import Hdf5DatasetLoader
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
from config import config
import skimage

In [2]:
data_path = "D:/development/tensorflow/datasets/alpr"

BATCH_SIZE = 12

In [3]:
loader = Hdf5DatasetLoader()
X_train, y_train = loader.load(data_path + '/hdf5/train.h5', shuffle=True)

In [28]:
data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=[0.8, 2.8],
    shear_range=2.5,
    brightness_range=[0.5, 1.5],
    fill_mode='nearest',
    horizontal_flip=False,
    validation_split=0.2)

images, labels = next(data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE))
images = [skimage.transform.resize(image, (config.IMAGE_HEIGHT, config.IMAGE_WIDTH)) for image in images]

def plotImages(images_arr, labels, rows, columns):
    fig, axes = plt.subplots(rows, columns, figsize=(50,50))
    axes = axes.flatten()
    for img, label, ax in zip(images_arr, labels, axes):
        #img = img.reshape(config.IMAGE_WIDTH, config.IMAGE_HEIGHT)
        ax.set_title(label)
        ax.imshow(img, cmap='gray')
    plt.tight_layout()
    plt.show()
    
plotImages(images, labels, int(BATCH_SIZE / 4), 4)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Andreas\Anaconda3\envs\dl4cv\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-b859e24a30d0>", line 13, in <module>
    images, labels = next(data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE))
  File "C:\Users\Andreas\Anaconda3\envs\dl4cv\lib\site-packages\keras_preprocessing\image\image_data_generator.py", line 432, in flow
    subset=subset
  File "C:\Users\Andreas\Anaconda3\envs\dl4cv\lib\site-packages\keras_preprocessing\image\numpy_array_iterator.py", line 117, in __init__
    'with shape', self.x.shape)
ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (65276, 32, 151))

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Andreas\Anaconda3\envs\dl4cv\lib\site-packages\IPython\core\interactiveshell.py"

ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (65276, 32, 151))